In [ ]:
# setup
! pip install bertopic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 4.1 MB/s 
     |████████████████████████████████| 662 kB 29.3 MB/s 
     |████████████████████████████████| 88 kB 6.1 MB/s 
     |████████████████████████████████| 85 kB 4.5 MB/s 
     |████████████████████████████████| 5.2 MB 24.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 5.8 MB 38.3 MB/s 
     |████████████████████████████████| 1.3 MB 49.2 MB/s 
     |████████████████████████████████| 182 kB 49.3 MB/s 
     |████████████████████████████████| 7.6 MB 51.6 MB/s 
     |████████████████████████████████| 1.1 MB 46.6 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp38-cp38-linux_x86_64.whl size=2700860 sha256=4dc8fa33d406a8825164c847520581d5ce94cc2150aedcb122e27cdb6fb03628
  Stored in directory: /root/.cache/

In [ ]:
#packages for making the model and setting up the data
from bertopic import BERTopic
import pandas as pd
import os

#packages for cleaning
import sys  
!{sys.executable} -m pip install contractions
import contractions  
import re  
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt') 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110 kB 5.4 MB/s 
     |████████████████████████████████| 287 kB 33.3 MB/s 


In [ ]:
#load in the data needed data
filepath_script = os.path.join("Data","all-scripts.csv")
filepath_date = os.path.join("Data","all-detailsepisodes.csv")
df_script = pd.read_csv("filepath_script")   # data contaning the scripts 
df_date = pd.read_csv("filepath_date") # data contaning the date of releace

#merging the two dataframes by episode id
df = pd.merge(df_script, df_date, on='episodeid')

#removing repeat colunm
df = df.drop('doctorid_y', axis=1)

In [ ]:
#getting rid of NAs, this also gets rid of things not tagged as talk, to check run: print(df['type'].unique())
df = df.dropna()  #319847 rows to 245618 

#making a function that removes special characters from the dates
def clean_date(x):
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@©[\\]^_`{|}~“…”’'), ' ', x)   #removes spetial charecteres
  return x

df["first_diffusion"] = df.first_diffusion.apply(clean_date)

In [ ]:
stop_words = stopwords.words('english')

#making a function that cleans the texts and tokenices it with NLTK
def clean_text(x):
  x = str(x)
  x = x.lower()
  x = contractions.fix(x)
  x = re.sub(r'#[A-Za-z0-9]*', ' ', x)   #removes induvial leters and numbers
  tokens = word_tokenize(x)
  x = ' '.join([w for w in tokens if not w.lower() in stop_words])
  x = re.sub(r'[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@©[\\]^_`{|}~“…”’'), ' ', x)   #removes spetial charecteres
  x = re.sub(r'[%s]' % re.escape("'"), ' ', x)  #removes '
  x = re.sub(r'\d+', ' ', x)     #removes Unicode character category [Nd]
  x = re.sub(r'\n+', ' ', x)     #removes new lines
  x = re.sub(r'\s{2,}', ' ', x)    #removes Unicode whitespace characters
  shortword = re.compile(r'\W*\b\w{1,2}\b') #removes words with only two letters
  x = shortword.sub('', x)
  return x

df['clean_text'] = df.text.apply(clean_text)

,idx,text,type,details,episodeid,doctorid_x,title,first_diffusion,clean_text
2,2,Where's mother?,talk,REMUS,21-7,6,The Twin Dilemma,22 Mar 1984,mother
3,3,She's busy.,talk,SYLVEST,21-7,6,The Twin Dilemma,22 Mar 1984,busy
4,4,Does that mean she isn't talking to us?,talk,ROMULUS,21-7,6,The Twin Dilemma,22 Mar 1984,mean talking
5,5,"No, she's just busy.",talk,SYLVEST,21-7,6,The Twin Dilemma,22 Mar 1984,busy
6,6,We would like to see her.,talk,BOTH,21-7,6,The Twin Dilemma,22 Mar 1984,would like see
...,...,...,...,...,...,...,...,...,...
319841,1198,You're expecting someone else?,talk,DOCTOR,21-6,5,The Caves of Androzani,8 Mar 1984,expecting someone else
319843,1200,"I, I, I.",talk,PERI,21-6,5,The Caves of Androzani,8 Mar 1984,
319844,1201,That's three I's in one breath. Makes you sou...,talk,DOCTOR,21-6,5,The Caves of Androzani,8 Mar 1984,three one breath makes sound ratheregotistical...
319845,1202,What's happened?,talk,PERI,21-6,5,The Caves of Androzani,8 Mar 1984,happened


In [ ]:
# put the data used for the model in list form, this is also needed for making a timeline
script = df.clean_text.to_list() 
date = df.first_diffusion.to_list()

Code for setting up a model:

In [ ]:
topic_model= BERTopic(language = "english")
topics, probs = topic_model.fit_transform(script)

In [ ]:
#reducing the number of topics to make it easier to work with:
topic_model.reduce_topics(script, nr_topics=1000)   

In [ ]:
#getting an overview of the topics generated
topic_model.get_topic_info() 

Different visualisations:

In [ ]:
topic_model.visualize_barchart(title = "Topic Word Score")

In [ ]:
#visualising the topics
#topic_model.visualize_topics()

In [ ]:
#topics_over_time = basemodel1.topics_over_time(script, date, nr_bins=20)
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=11) 

In [ ]:
#code for finding related topics, here looking fro the word "war" :
topic_model.find_topics("war", top_n=5) 

([191, 135, 100, 352, 152],
 [0.9414518153964146,
  0.7126270719014002,
  0.6834460056995192,
  0.6053331224358521,
  0.6043906971958998])